# Imports

In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass
import torchvision
from torchvision.models import densenet161, DenseNet161_Weights, vit_b_16, ViT_B_16_Weights, densenet121, DenseNet121_Weights
import os
import sys
from pathlib import Path
from torchinfo import summary

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
from CheXpert.race_prediction.dataset import CheXpertRaceDataset
from CheXpert.disease_prediction.dataset import CheXpertDiseaseDataset
from shared_utils import vprint, to_gpu
import shared_utils
from CheXpert.disease_prediction.utils import Configs as disease_configs
from CheXpert.race_prediction.utils import Configs as race_configs

# Configs

In [4]:
@dataclass
class Configs:
    DATA_DIR = os.path.join("data", "CheXpert", "CheXpert-v1.0-small")
    CXR_DATA_DIR = os.path.join("data", "MIMIC-CXR-JPG")
    DISEASE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "disease_prediction", "trained_models")
    RACE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "race_prediction", "trained_models")
    VALID_LABELS_FILENAME = "valid.csv"
    DEMO_FILENAME = "CHEXPERT DEMO.csv"
    DISEASE_CLASSES = disease_configs.ANNOTATIONS_COLUMNS
    RACE_CLASSES = race_configs.ANNOTATIONS_COLUMNS
    NUM_DISEASE_CLASSES = disease_configs.NUM_CLASSES
    NUM_RACE_CLASSES = race_configs.NUM_CLASSES
    BATCH_SIZE = 4
    SEED = 123
    VERBOSE = 1
    VALID_SIZE_DEBUG = -1

In [5]:
shared_utils.set_seed(Configs.SEED)

In [6]:
if torch.cuda.is_available():
    vprint(f"Memory info: {torch.cuda.mem_get_info()[0]/10e8:.1f} GB free GPU.", Configs)
else: 
    vprint(f"No GPU Memory.", Configs)

2022-07-22 19:01: No GPU Memory.


In [7]:
valid_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Disease Prediction 

In [8]:
disease_valid_dataset = CheXpertDiseaseDataset(data_dir=Configs.DATA_DIR, 
                                               labels_filename=Configs.VALID_LABELS_FILENAME,
                                               transform=valid_transform)
disease_valid_dataset.df_labels = disease_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG] # hack for speed debugging
disease_valid_dataloader = DataLoader(disease_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(disease_valid_dataset)

233

In [9]:
_, _, files = next(os.walk(Configs.DISEASE_TRAINED_MODELS_DIR))
disease_trained_models = [os.path.join(Configs.DISEASE_TRAINED_MODELS_DIR, file) for file in files]
len(disease_trained_models)

1

In [10]:
disease_model = densenet121()
num_features = disease_model.classifier.in_features
disease_model.classifier = nn.Sequential(
    nn.Linear(num_features, num_features, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=num_features, out_features=Configs.NUM_DISEASE_CLASSES, bias=True)
)
disease_model.eval()
not disease_model.training

True

In [11]:
disease_model, results, _, _ = shared_utils.load_statedict(disease_model, disease_trained_models[0], Configs)
disease_model = to_gpu(disease_model)

2022-07-22 19:01: Loading model - CheXpert\disease_prediction\trained_models\2022_07_12-18_47__densenet121_aug__epoch-5__iter-12659__batch_size-16__trainLastLoss-0.3754__validAUC-0.8899.dict


In [12]:
disease_labels, disease_outputs = shared_utils.get_metric_tensors(disease_model, disease_valid_dataloader, Configs,
                                                  apply_on_outputs=lambda x: torch.sigmoid(x),
                                                  by_study=False, challenge_ann_only=None)

KeyboardInterrupt: 

In [59]:
df_res_disease = pd.DataFrame(columns=Configs.DISEASE_CLASSES, 
                              data=[shared_utils.auc_score(disease_labels, disease_outputs, per_class=True),],
                             index=['AUC'])
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
AUC,0.837498,0.833868,0.932727,0.939007,0.936763


# Race Prediction

In [21]:
# Create data loaders.
race_valid_dataset = CheXpertRaceDataset(data_dir=Configs.DATA_DIR, demo_filename=Configs.DEMO_FILENAME,
                                       labels_filename=Configs.VALID_LABELS_FILENAME,
                                       transform=valid_transform)
race_valid_dataloader = DataLoader(race_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
Configs.TRAIN_LOADER_SIZE = len(race_valid_dataloader)
len(race_valid_dataset)

172

In [22]:
_, _, files = next(os.walk(Configs.RACE_TRAINED_MODELS_DIR))
race_trained_models = [os.path.join(Configs.RACE_TRAINED_MODELS_DIR, file) for file in files]
race_model_versions = [p.split('__')[1] for p in race_trained_models]
len(race_trained_models)

8

In [23]:
df_res_race = pd.DataFrame(columns=Configs.RACE_CLASSES)
df_res_race

,White,Asian,Black,Hispanic


In [24]:
for model_version, model_path in zip(race_model_versions, race_trained_models):
    model = densenet121()
    
    if "shallow" in model_version:
        shallow_denseblock = int(model_version.split('_')[2][10:])
        layer_offset = 3 + 2 * shallow_denseblock
        num_features = model.features[layer_offset].norm.num_features
        model = model.features[:layer_offset]
        classifier_module = nn.Sequential(
            nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),   
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=num_features, out_features=num_features, bias=True),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
        model.add_module('classifier', classifier_module)
    else:
        num_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, num_features, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
    model.eval()
    model = to_gpu(model)
    model, results, _, _ = shared_utils.load_statedict(model, model_path, Configs)
    labels, outputs = shared_utils.get_metric_tensors(model, race_valid_dataloader, Configs,
                                                      apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                      by_study=False, challenge_ann_only=None)
    auc_scores = shared_utils.auc_score(labels, outputs, per_class=True)
    df_res_race.loc[model_version] = auc_scores

df_res_race   

2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-15_29__densenet121_race_denseblock1_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2805__validAUC-0.9178.dict
2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-02_27__densenet121_race_denseblock2_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2811__validAUC-0.9094.dict
2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-10_11__densenet121_race_classifier_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.732__validAUC-0.6978.dict
2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_21-13_21__densenet121_race_denseblock2_shallow__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.3791__validAUC-0.9075.dict
2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-21_46__densenet121_race_denseblock3_freezed__epoch-9__iter-10266__batch_size-16__

,White,Asian,Black,Hispanic
densenet121_race_denseblock1_freezed,0.922287,0.895706,0.955357,0.897821
densenet121_race_denseblock2_freezed,0.904110,0.873892,0.973214,0.886447
densenet121_race_classifier_freezed,0.796628,0.583504,0.666667,0.744361
densenet121_race_denseblock2_shallow,0.882508,0.909339,0.944940,0.893195
densenet121_race_denseblock3_freezed,0.898841,0.805726,0.959821,0.876807
densenet121_race_denseblock4_freezed,0.893836,0.775733,0.937500,0.841334
densenet121_race_denseblock1_shallow,0.919916,0.799591,0.888393,0.868517
densenet121_race,0.904900,0.934560,0.958333,0.880663


In [28]:
df_res_race["mean"] = df_res_race.mean(axis=1)
df_res_race.sort_values(by="mean", ascending=False, inplace=True)
df_res_race = df_res_race.round(2)

In [29]:
df_res_race

,White,Asian,Black,Hispanic,mean
densenet121_race,0.90,0.93,0.96,0.88,0.92
densenet121_race_denseblock1_freezed,0.92,0.90,0.96,0.90,0.92
densenet121_race_denseblock2_freezed,0.90,0.87,0.97,0.89,0.91
densenet121_race_denseblock2_shallow,0.88,0.91,0.94,0.89,0.91
densenet121_race_denseblock3_freezed,0.90,0.81,0.96,0.88,0.89
densenet121_race_denseblock1_shallow,0.92,0.80,0.89,0.87,0.87
densenet121_race_denseblock4_freezed,0.89,0.78,0.94,0.84,0.86
densenet121_race_classifier_freezed,0.80,0.58,0.67,0.74,0.70


# Disease Per Race Performance

In [55]:
df_race = race_valid_dataset.df_labels[race_valid_dataset.df_labels.columns[:10]].drop_duplicates()
df_joined = disease_valid_dataset.df_labels.merge(df_race, how='left', left_on='patient_id', right_on='PATIENT')
df_joined.race.fillna('Other', inplace=True) 
df_joined.head(2)

,original_path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,img_path,patient_id,study,view,PATIENT,GENDER,AGE_AT_CXR,PRIMARY_RACE,ETHNICITY,race,Asian,Black,Hispanic,White
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64541,study1,view1_frontal.jpg,patient64541,Male,73.0,White,Non-Hispanic/Non-Latino,White,0.0,0.0,0.0,1.0
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64542,study1,view1_frontal.jpg,patient64542,Male,70.0,White,Non-Hispanic/Non-Latino,White,0.0,0.0,0.0,1.0


In [61]:
df_joined.race.value_counts()

White       132
Other        62
Asian        26
Black         9
Hispanic      4
Name: race, dtype: int64

In [60]:
for race in df_joined.race.unique():
    disease_race_labels = disease_labels[df_joined.race==race]
    disease_race_outputs = disease_outputs[df_joined.race==race]
    df_res_disease.loc[race] = shared_utils.auc_score(disease_race_labels, disease_race_outputs, per_class=True)
df_res_disease

Only one class present in y_true. ROC AUC score is not defined in that case.


,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
AUC,0.837498,0.833868,0.932727,0.939007,0.936763
White,0.852639,0.820924,0.935650,0.941308,0.922452
Asian,0.803922,0.849673,0.886364,1.000000,0.921569
Other,0.776786,0.836310,0.928571,0.909615,0.983007
Hispanic,1.000000,0.750000,1.000000,-1.000000,1.000000
Black,1.000000,1.000000,1.000000,0.900000,1.000000


In [16]:
df_disease = pd.DataFrame(columns=Configs.DISEASE_CLASSES, 
                              data=[shared_utils.auc_score(disease_labels, disease_outputs, per_class=True),],
                              index=['all_races'])
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
all_races,0.837498,0.833868,0.932727,0.939007,0.936763


# MIMIC CXR JPG

In [63]:
df_adm = pd.read_csv(os.path.join(Configs.CXR_DATA_DIR, "admissions.csv"))
display(df_adm.shape)
display(df_adm.head(2))

(523740, 15)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0


In [64]:
df_labels = pd.read_csv(os.path.join(Configs.CXR_DATA_DIR, "mimic-cxr-2.0.0-chexpert.csv"))
display(df_labels.shape)
display(df_labels.head(2))

(227827, 16)

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [145]:
df_split = pd.read_csv(os.path.join(Configs.CXR_DATA_DIR, "mimic-cxr-2.0.0-split.csv"))
display(df_split.shape)
display(df_split.head(2))

(377110, 4)

,dicom_id,study_id,subject_id,split
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,50414267,10000032,train
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,50414267,10000032,train


In [144]:
df_patients = pd.read_csv(os.path.join(Configs.CXR_DATA_DIR, "patients.csv"))
display(df_patients.shape)
display(df_patients.head(2))

(315460, 6)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN


In [148]:
df_cxr_joined = df_labels.merge(df_adm, on='subject_id').merge(df_patients, on='subject_id').merge(df_split, on=['subject_id', 'study_id'])
df_cxr_joined.head(2)

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,dicom_id,split
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0,F,52,2180,2014 - 2016,2180-09-09,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,train
1,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0,F,52,2180,2014 - 2016,2180-09-09,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,train


In [149]:
race_dict = {
    "WHITE": "White",
    "OTHER": "Other",
    "ASIAN": "Asian",
    "BLACK/AFRICAN AMERICAN": "Black",
    "HISPANIC/LATINO": "Hispanic",
    "UNKNOWN": "Other",
    "UNABLE TO OBTAIN": "Other",
    "AMERICAN INDIAN/ALASKA NATIVE": "Other"
}
df_cxr_joined['race'] = df_cxr_joined.ethnicity.replace(race_dict)

In [150]:
def age_to_age_group(age):
    if age < 40:
        return '20-40'
    elif age < 70:
        return '40-70'
    return '70-90'


df_cxr_joined['age'] = df_cxr_joined.anchor_age.apply(age_to_age_group)

In [151]:
df_cxr_demo = df_cxr_joined[['subject_id', 'study_id', 'split'] + ['race', 'age', 'gender'] + Configs.DISEASE_CLASSES]
df_cxr_demo.head(2)

,subject_id,study_id,split,race,age,gender,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
0,10000032,50414267,train,White,40-70,F,NaN,NaN,NaN,NaN,NaN
1,10000032,50414267,train,White,40-70,F,NaN,NaN,NaN,NaN,NaN


In [155]:
df_temp = (df_cxr_demo.groupby('subject_id')[['race', 'age', 'gender']].nunique() == 1).all(axis=1)
valid_subject_ids = df_temp[df_temp.values].index
df_cxr_demo = df_cxr_demo[df_cxr_demo.subject_id.isin(valid_subject_ids)].drop_duplicates()
display(df_cxr_demo.shape)
display(df_cxr_demo.head(2))

(196609, 11)

,subject_id,study_id,split,race,age,gender,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
0,10000032,50414267,train,White,40-70,F,NaN,NaN,NaN,NaN,NaN
8,10000032,53189527,train,White,40-70,F,NaN,NaN,NaN,NaN,NaN


In [156]:
df_cxr_demo.groupby(['race', 'age', 'gender']).subject_id.count()

race      age    gender
Asian     20-40  F           519
                 M           400
          40-70  F          1368
                 M          1965
          70-90  F          1259
                 M          1426
Black     20-40  F          2898
                 M          1973
          40-70  F         11385
                 M          7639
          70-90  F          5576
                 M          2910
Hispanic  20-40  F          1040
                 M          1238
          40-70  F          3134
                 M          3103
          70-90  F          1278
                 M           807
Other     20-40  F           922
                 M          1825
          40-70  F          2787
                 M          4457
          70-90  F          2318
                 M          2532
White     20-40  F          5587
                 M          5436
          40-70  F         28999
                 M         38577
          70-90  F         26273
                 M 

In [157]:
cxr_table = pd.pivot_table(df_cxr_demo, values='subject_id', index=['gender', 'age'],
                    columns=['race'], aggfunc='count', fill_value=0)
cxr_table

race          Asian  Black  Hispanic  Other  White
gender age                                        
F      20-40    519   2898      1040    922   5587
       40-70   1368  11385      3134   2787  28999
       70-90   1259   5576      1278   2318  26273
M      20-40    400   1973      1238   1825   5436
       40-70   1965   7639      3103   4457  38577
       70-90   1426   2910       807   2532  26978

In [159]:
cxr_table = pd.pivot_table(df_cxr_demo[df_cxr_demo.split.isin(['test', 'validate'])], values='subject_id', index=['gender', 'age'],
                    columns=['race'], aggfunc='count', fill_value=0)
cxr_table

race          Asian  Black  Hispanic  Other  White
gender age                                        
F      20-40      2     23         8      1     57
       40-70     24    307        15      8    795
       70-90     47    125        45     58    559
M      20-40      0      6         8    101     69
       40-70     25    230        94     72    871
       70-90     33     85         2     24    721

In [160]:
df_cxr_demo.groupby(['race', 'age', 'gender']).sample(n=200, replace=False, random_state=Configs.SEED)

,subject_id,study_id,split,race,age,gender,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
1399230,15377769,52948601,train,Asian,20-40,F,NaN,NaN,NaN,NaN,NaN
1692110,16509312,55986691,train,Asian,20-40,F,NaN,NaN,NaN,NaN,NaN
1547294,15946579,56651565,train,Asian,20-40,F,NaN,NaN,NaN,NaN,NaN
723106,12780512,51955339,train,Asian,20-40,F,-1.0,NaN,-1.0,NaN,1.0
1692108,16509312,51062733,train,Asian,20-40,F,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2228617,18477137,58315993,train,White,70-90,M,NaN,1.0,NaN,NaN,NaN
564003,12189597,51962902,train,White,70-90,M,NaN,NaN,NaN,NaN,NaN
2550045,19664531,54975400,train,White,70-90,M,-1.0,NaN,NaN,NaN,NaN
585083,12287689,54410594,train,White,70-90,M,NaN,NaN,1.0,NaN,NaN
